In [1]:
#%load_ext autoreload
#%autoreload 2

In [2]:
from sklearn.model_selection import train_test_split
from functools import *
from datetime import datetime
import sys
import os

PROJECT_ROOT = os.path.abspath(os.getcwd() + os.sep + os.pardir)
sys.path.insert(0, PROJECT_ROOT)

from IRESNs_tensorflow.time_series_datasets import *
from IRESNs_tensorflow.models import *
from benchmarks import *
import notify2  # TODO replace notify watch here : https://notify2.readthedocs.io/en/latest/
from tensorflow import keras

DATA_ROOT = os.path.join(PROJECT_ROOT, "datasets")
TUNER_ROOT = os.path.join(PROJECT_ROOT, "models")
BENCHMARKS_ROOT = os.path.join(PROJECT_ROOT, "plots", "benchmarks")
WEIGHTS_ROOT = os.path.join(PROJECT_ROOT, "plots", "weights")
TB_ROOT = os.path.join(os.path.abspath(os.sep), "tmp", "tensorboard")

BENCHMARKS_DIR = "benchmarks"
WEIGHTS_DIR = "weights"

print("       Data dir:", DATA_ROOT)
print("      Tuner dir:", TUNER_ROOT)
print("Tensorboard dir:", TB_ROOT)

TUNER = "RandomSearch"  # "Hyperband" or "BayesianOptimization" or "RandomSearch"

MAX_TRIALS = 500  # How many configuration the tuner will try
MAX_EPOCHS = 500  # How many epochs the tuner train the model for each trials
TRIALS = 1  # Positive Integer. How many iterations for one set of hyperparameters
PATIENCE = 10 # EarlyStopping
BENCHMARKS_TRIALS = 10  # How many times do the benchmark. 0 to skip BENCHMARKS

MAX_UNITS = 250
UNITS = sorted([50, 75, 100, 150, 250])
MINVAL = 0.01
MAXVAL = 1.5

SKIP = True  # Skip if a model is already tested?
OVERWRITE = False  # Redo the model selection for a model?

READOUT_ACTIVATION_BINARY = keras.activations.sigmoid
LOSS_FUNCTION_BINARY = keras.losses.BinaryCrossentropy()
READOUT_ACTIVATION = keras.activations.softmax  # https://www.tensorflow.org/api_docs/python/tf/keras/activations
LOSS_FUNCTION = keras.losses.SparseCategoricalCrossentropy()  # https://www.tensorflow.org/api_docs/python/tf/keras/losses

if not os.path.exists(TUNER_ROOT):
    os.makedirs(TUNER_ROOT)
if not os.path.exists(TB_ROOT):
    os.makedirs(TB_ROOT)

TUNER_STRING = TUNER + "." + str(MAX_EPOCHS) + "me" + str(MAX_TRIALS) + "mt.testFinale"

benchmarks = BenchmarksDB(load_path=os.path.join(BENCHMARKS_ROOT, "benchmarks."+ TUNER_STRING +".json"), plot_path=WEIGHTS_ROOT)

/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/sktime/datatypes/_series/_check.py:42: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex

       Data dir: /dati/luca/Uni-Luca/Tesi/progetto/datasets
      Tuner dir: /dati/luca/Uni-Luca/Tesi/progetto/models
Tensorboard dir: /tmp/tensorboard



|        | ArticularyWordRecognition | CharacterTrajectories | Epilepsy | JapaneseVowels  | Libras | SpokenArabicDigits |
|--------|:-------------------------:|:---------------------:|:--------:|:---------------:|:------:|:------------------:|
| Input  |             9             |           3           |    3     |       12        |   2    |         13         |
| Output |            25             |          20           |    4     |        9        |   15   |         10         |

In [3]:
from time import time
from keras_tuner import Hyperband, BayesianOptimization, RandomSearch
from keras import callbacks as kc

class CurrentMSModel(kc.Callback):
    def __init__(self, names):
        super().__init__()
        self.dataset_name, self.class_name, self.experiment_name, self.model_name = names

    def on_train_begin(self, logs=None):
        #ids.clear_output(wait=True)
        print("MS of {} {} {} {}".format(self.dataset_name, self.class_name, self.experiment_name, self.model_name))


import hashlib
def get_seed(exp_name, mod_name):
    union = exp_name + mod_name
    a = hashlib.md5(union.encode('UTF-8'))
    return int(a.hexdigest(), 16)


def model_selection(build_model_fn, names,
                    train_set, val_set,
                    tuner_path, verbose=1):
    dataset_name, class_name, experiment_name, model_name = names
    x_train, y_train = train_set
    x_val, y_val = val_set
    seed = get_seed(experiment_name, model_name)
    if TUNER == "Hyperband":
        working_dir = os.path.join(tuner_path, TUNER_STRING, dataset_name, class_name)
        if not os.path.exists(working_dir):
            os.makedirs(working_dir)

        tuner = Hyperband(
            build_model_fn,
            objective='val_accuracy',
            max_epochs=MAX_EPOCHS,
            hyperband_iterations=1.,
            seed=seed,
            directory=working_dir,
            project_name=experiment_name + ' ' + model_name,
            overwrite=OVERWRITE,
            executions_per_trial=TRIALS,
        )
    elif TUNER == "BayesianOptimization":
        working_dir = os.path.join(tuner_path, TUNER_STRING, dataset_name, class_name)
        if not os.path.exists(working_dir):
            os.makedirs(working_dir)

        tuner = BayesianOptimization(
            build_model_fn,
            objective='val_accuracy',
            max_trials=MAX_TRIALS,
            seed=seed,

            directory=working_dir,
            project_name=experiment_name + ' ' + model_name,
            overwrite=OVERWRITE,
            executions_per_trial=TRIALS,
        )
    elif TUNER == "RandomSearch":
        working_dir = os.path.join(tuner_path, TUNER_STRING, dataset_name, class_name)
        if not os.path.exists(working_dir):
            os.makedirs(working_dir)

        tuner = RandomSearch(
            build_model_fn,
            objective='val_accuracy',
            max_trials=MAX_TRIALS,
            seed=seed,

            directory=working_dir,
            project_name=experiment_name + ' ' + model_name,
            overwrite=OVERWRITE,
            executions_per_trial=TRIALS,
        )
    else:
        raise ValueError("Unknown Tuner -> {}".format(TUNER))

    # now the tuner will search the best hyperparameters
    tuner.search(x_train, y_train, epochs=MAX_EPOCHS, validation_data=(x_val, y_val),
                 callbacks=[
                     keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE),
                     #CurrentMSModel(names)
                 ], verbose=verbose)

    return tuner, tuner.oracle.get_best_trials(1)[0].score

def testing_model(names, tuner,
                  train_set, val_set, test_set,
                  tensorboard_path=None, benchmarks_verbose=0):
    dataset_name, class_name, experiment_name, model_name = names
    x_train, y_train = train_set
    x_val, y_val = val_set
    x_test, y_test = test_set

    # keras.callbacks.CallbackList([])
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True)]
    if tensorboard_path is not None:
        tensorboard_dir = tensorboard_path + model_name
        callbacks.append(keras.callbacks.TensorBoard(tensorboard_dir, profile_batch='500,500'))

    print("[{}] Running {} benchmarks".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S"), BENCHMARKS_TRIALS))

    best_model_hp = tuner.get_best_hyperparameters()[0]

    test_model = None

    required_time = []
    train_acc = []
    val_acc = []
    test_acc = []


    tf.random.set_seed(int(hash((experiment_name, model_name))))

    for i in range(BENCHMARKS_TRIALS):
        initial_time = time()

        test_model = tuner.hypermodel.build(best_model_hp)
        history = test_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=MAX_EPOCHS,
                       callbacks=callbacks, verbose=benchmarks_verbose)
        test_loss, accuracy = test_model.evaluate(x_test, y_test)

        required_time.append(time() - initial_time)

        train_acc.append(history.history['accuracy'][-1])
        val_acc.append(history.history['val_accuracy'][-1])
        test_acc.append(accuracy)

    stat = Statistic(best_model_hp, train_acc, val_acc, test_acc, required_time)

    return test_model, stat


notify2_init = False

def send_notification(title, message):
    def notify2_init_fun():
        global notify2_init
        if not notify2_init:
            notify2.init("Tesi")

    notify2_init_fun()

    notice = notify2.Notification(title, message)
    notice.show()


# Local Hyperparameters

In [4]:
# This cell is a little nightmare

class HP:
    FREE = 0
    FIXED = 1
    RESTRICTED = 2

    def __init__(self, hp_type, value=None):
        self.value = value
        self.type = hp_type

    @classmethod
    def free(cls):
        return cls(HP.FREE)

    @classmethod
    def fixed(cls, value):
        return cls(HP.FIXED, value)

    @classmethod
    def restricted(cls, value=None):
        return cls(HP.RESTRICTED, value)


def get_units(tuner, name, hp, min_value, max_value, sampling=None):
    if hp.type == HP.FREE:
        tmp = tuner.Int(name, min_value=min_value, max_value=max_value, sampling=sampling)
    elif hp.type == HP.RESTRICTED:
        tmp = tuner.Choise(name, UNITS, ordered=True)
    elif hp.type == HP.FIXED:
        tmp = tuner.Fixed(name, hp.value)
    else:
        raise ValueError("HP type not found")
    return tmp

def get_float(tuner, name, hp, min_value, max_value, sampling=None, pn=None, pv=None):
    if hp.type == HP.FREE or (hp.type == HP.RESTRICTED and hp.value is None):
        tmp = tuner.Float(name, min_value=min_value, max_value=max_value, sampling=sampling, parent_name=pn,
                          parent_values=pv)
    elif hp.type == HP.FIXED or (hp.type == HP.RESTRICTED and hp.value is not None):
        tmp = tuner.Fixed(name, hp.value)
    else:
        raise ValueError("HP type not found")
    return tmp

def get_bool(tuner, name, hp):
    if hp.type == HP.FREE or hp.type == HP.RESTRICTED:
        tmp = tuner.Boolean(name)
    elif hp.type == HP.FIXED:
        tmp = tuner.Fixed(name, hp.value)
    else:
        raise ValueError("HP type not found")
    return tmp

def get_float_vec(tuner, name, hp, length, min_value, max_value, step=None, sampling=None, pn=None, pv=None):
    if hp.type == HP.FREE:
        tmp = [tuner.Float(name + ' ' + str(i), min_value=min_value, max_value=max_value, sampling=sampling,
                           parent_name=pn, parent_values=pv, step=step)
               for i in range(length)]
    elif hp.type == HP.FIXED:
        tmp = [tuner.Fixed(name + ' ' + str(i), hp.value[i]) for i in range(length)]
    elif hp.type == HP.RESTRICTED:
        tmp2 = tuner.Float(name + ' 0', min_value=min_value, max_value=max_value, sampling=sampling, parent_name=pn,
                               parent_values=pv, step=step)
        tmp = [tmp2 for _ in range(length)]
    else:
        raise ValueError("HP type not found")
    return tmp

def get_connectivity_esn1(tuner, hp):
    if hp.type == HP.FREE:
        tmp = tuner.Float('connectivity 0', min_value=0.1, max_value=1., sampling="linear")
    elif hp.type == HP.FIXED:
        raise ValueError("HP.FIXED not valid for connectivity")
    elif hp.type == HP.RESTRICTED:
        tmp = tuner.Fixed('connectivity 0', hp.value)
    else:
        raise ValueError("HP type not found")
    return tmp

def get_connectivity_esn2(tuner, hp, length):
    if hp.type == HP.FREE:
        tmp = [tuner.Float('connectivity ' + str(i), min_value=0., max_value=1., sampling="linear") for i in range(length)]
    elif hp.type == HP.FIXED:
        raise ValueError("HP.FIXED not valid for connectivity")
    elif hp.type == HP.RESTRICTED:
        if hp.value is None:
            tmp2 = tuner.Float('connectivity 0', min_value=0., max_value=1., sampling="linear")
        else:
            tmp2 = tuner.Fixed('connectivity 0', hp.value)
        tmp = [tmp2 for _ in range(length)]
    else:
        raise ValueError("HP type not found")
    return tmp

def get_connectivity(tuner, hp, length):  # It is a squared matrix
    if hp.type == HP.FREE:
        conn_matrix = [[tuner.Float('connectivity ' + str(i), min_value=0., max_value=1., sampling="linear") if i == j else
                        tuner.Float('connectivity ' + str(i) + '->' + str(j), min_value=0., max_value=1., sampling="linear")
                        for i in range(length)]
                       for j in range(length)]
    elif hp.type == HP.FIXED:
        diagonal, off_diagonal = hp.value
        off_diagonal = tuner.Fixed('connectivity X->Y', off_diagonal)
        conn_matrix = [[tuner.Fixed('connectivity ' + str(i), diagonal) if i == j else
                        off_diagonal
                        for i in range(length)]
                       for j in range(length)]
    elif hp.type == HP.RESTRICTED:
        if hp.value is None:
            connectivity = [tuner.Float('connectivity ' + str(i), min_value=0., max_value=1., sampling="linear") for i in range(length)]
        else:
            tmp = tuner.Fixed('connectivity 0', hp.value)
            connectivity = [ tmp for _ in range(length)]
        intra_connectivity = tuner.Float('connectivity X->Y', min_value=0., max_value=1., sampling="linear")
        conn_matrix = [[connectivity[i] if i == j else intra_connectivity for i in range(length)] for j in range(length)]
    else:
        raise ValueError("HP type not found")
    return conn_matrix

def get_minmax(tuner, hp, length):  # Is a matrix for ESN3 and ESN4
    if hp.type == HP.FREE:
        minmax_vec = [[0. if i == j else tuner.Float('minmax ' + str(i) + '->' + str(j), min_value=MINVAL, max_value=MAXVAL, sampling="linear")
                   for i in range(length)]
                  for j in range(length)]
    elif hp.type == HP.RESTRICTED:
        minmax = tuner.Float('minmax', min_value=MINVAL, max_value=MAXVAL, sampling="linear")
        minmax_vec = [[0. if i == j else minmax
                   for i in range(length)]
                  for j in range(length)]
    elif hp.type == HP.FIXED:
        minmax = hp.value
        minmax_vec = [[0. if i == j else minmax
                   for i in range(length)]
                  for j in range(length)]
    else:
        raise ValueError("HP type not found")
    return minmax_vec

def get_spectral_radius(tuner, hp, length):  # Is a vector
    if hp.type == HP.FREE:
        sr_vec = [tuner.Float('spectral radius ' + str(i), min_value=MINVAL, max_value=MAXVAL, sampling="linear") for i in range(length)]
    elif hp.type == HP.RESTRICTED:
        spectral_radius = tuner.Float('spectral radius 0', min_value=MINVAL, max_value=MAXVAL, sampling="linear")
        sr_vec = [spectral_radius for _ in range(length)]
    elif hp.type == HP.FIXED:
        spectral_radius = hp.value
        sr_vec = [spectral_radius for _ in range(length)]
    else:
        raise ValueError("HP type not found")
    return sr_vec

def get_gsr(tuner, gsr):
    global_sr = get_bool(tuner, 'use G.S.R.', gsr)
    if global_sr:
        global_sr = tuner.Float('G.S.R.', min_value=MINVAL, max_value=MAXVAL, sampling="linear", parent_name='use G.S.R.', parent_values=True)
    else:
        global_sr = None

    return global_sr


# Build model functions

In [5]:
#

def build_ESN(output_classes, _reservoirs,  # Defined by dataset
               units, spectral_radius, _gsr, _off_diag, connectivity, input_scaling, bias_scaling, leaky, learning_rate,  # Defined by experiment
               tuner) -> ESN:
    if output_classes == 2:
        output_units = 1
        readout_activation = READOUT_ACTIVATION_BINARY
        loss = LOSS_FUNCTION_BINARY
    else:
        output_units = output_classes
        readout_activation = READOUT_ACTIVATION
        loss = LOSS_FUNCTION

    tmp_model = ESN(units=get_units(tuner, 'units', units, 50, MAX_UNITS),
                     connectivity=get_connectivity_esn1(tuner, connectivity),
                     spectral_radius=get_float(tuner, 'spectral radius 0', spectral_radius, min_value=MINVAL,
                                               max_value=MAXVAL, sampling="linear"),
                     output_units=output_units,
                     readout_activation=readout_activation,
                     input_scaling=get_float(tuner, 'input scaling 0', input_scaling, min_value=MINVAL,
                                             max_value=MAXVAL, sampling="linear"),
                     bias_scaling=get_float(tuner, 'bias scaling 0', bias_scaling, min_value=MINVAL, max_value=MAXVAL,
                                            sampling="linear"),
                     leaky=get_float(tuner, 'leaky', leaky, min_value=0.0, max_value=1., sampling="linear"),
                     )

    alpha = get_float(tuner, 'learning rate', learning_rate, min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.Adam(alpha),
        loss=loss,
        metrics=['accuracy'],
    )
    return tmp_model


def build_IRESN(output_classes, reservoirs,  # Defined by dataset
               units, spectral_radius, gsr, _off_diag, connectivity, input_scaling, bias_scaling, leaky, learning_rate,
               # Defined by experiment
               tuner) -> IRESN:
    if output_classes == 2:
        output_units = 1
        readout_activation = READOUT_ACTIVATION_BINARY
        loss = LOSS_FUNCTION_BINARY
    else:
        output_units = output_classes
        readout_activation = READOUT_ACTIVATION
        loss = LOSS_FUNCTION

    tmp_model = IRESN(units=get_units(tuner, 'units', units, 50, MAX_UNITS),
                     sub_reservoirs=reservoirs,
                     connectivity=get_connectivity_esn2(tuner, connectivity, reservoirs),
                     spectral_radius=get_spectral_radius(tuner, spectral_radius, reservoirs),
                     gsr=get_gsr(tuner, gsr),
                     output_units=output_units,
                     readout_activation=readout_activation,
                     input_scaling=get_float(tuner, 'input scaling 0', input_scaling, min_value=MINVAL,
                                             max_value=MAXVAL, sampling="linear"),
                     bias_scaling=get_float(tuner, 'bias scaling 0', bias_scaling, min_value=MINVAL, max_value=MAXVAL,
                                            sampling="linear"),
                     leaky=get_float(tuner, 'leaky', leaky, min_value=0.0, max_value=1., sampling="linear"),
                     )

    alpha = get_float(tuner, 'learning rate', learning_rate, min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=loss,
        metrics=['accuracy'],
    )
    return tmp_model


def build_IIRESN(output_classes, reservoirs,  # Defined by dataset
               units, spectral_radius, gsr, off_diag, connectivity, input_scaling, bias_scaling, leaky, learning_rate,
               # Defined by experiment
               tuner) -> IIRESN:

    if output_classes == 2:
        output_units = 1
        readout_activation = READOUT_ACTIVATION_BINARY
        loss = LOSS_FUNCTION_BINARY
    else:
        output_units = output_classes
        readout_activation = READOUT_ACTIVATION
        loss = LOSS_FUNCTION

    tmp_model = IIRESN(units=get_units(tuner, 'units', units, 50, MAX_UNITS),
                     sub_reservoirs=reservoirs,
                     connectivity=get_connectivity(tuner, connectivity, reservoirs),
                     spectral_radius=get_spectral_radius(tuner, spectral_radius, reservoirs),
                     gsr=get_gsr(tuner, gsr),
                     off_diagonal_limits=get_minmax(tuner, off_diag, reservoirs),
                     output_units=output_units,
                     readout_activation=readout_activation,
                     input_scaling=get_float(tuner, 'input scaling 0', input_scaling, min_value=MINVAL,
                                             max_value=MAXVAL, sampling="linear"),
                     bias_scaling=get_float(tuner, 'bias scaling 0', bias_scaling, min_value=MINVAL, max_value=MAXVAL,
                                            sampling="linear"),
                     leaky=get_float(tuner, 'leaky', leaky, min_value=0.0, max_value=1., sampling="linear"),
                     )

    alpha = get_float(tuner, 'learning rate', learning_rate, min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=loss,
        metrics=['accuracy'],
    )
    return tmp_model


def build_IIRESNvsr(output_classes, reservoirs,  # Defined by dataset
               units, spectral_radius, gsr, off_diag, connectivity, input_scaling, bias_scaling, leaky, learning_rate,
               # Defined by experiment
               tuner) -> IIRESNvsr:

    if output_classes == 2:
        output_units = 1
        readout_activation = READOUT_ACTIVATION_BINARY
        loss = LOSS_FUNCTION_BINARY
    else:
        output_units = output_classes
        readout_activation = READOUT_ACTIVATION
        loss = LOSS_FUNCTION

    partitions = [tuner.Float('partition ' + str(i), min_value=0.1, max_value=1.0, sampling="linear") for i in range(reservoirs)]
    total = sum(partitions)
    # Normalize the partition vector now sum(partitions) == 1.
    partitions = list(map(lambda _x: 0 if total == 0 else _x / total, partitions))

    tmp_model = IIRESNvsr(units=get_units(tuner, 'units', units, 50, MAX_UNITS),
                     sub_reservoirs=reservoirs,
                     connectivity=get_connectivity(tuner, connectivity, reservoirs),
                     partitions=partitions,
                     spectral_radius=get_spectral_radius(tuner, spectral_radius, reservoirs),
                     gsr=get_gsr(tuner, gsr),
                     off_diagonal_limits=get_minmax(tuner, off_diag, reservoirs),
                     output_units=output_units,
                     readout_activation=readout_activation,
                     input_scaling=get_float(tuner, 'input scaling 0', input_scaling, min_value=MINVAL,
                                             max_value=MAXVAL, sampling="linear"),
                     bias_scaling=get_float(tuner, 'bias scaling 0', bias_scaling, min_value=MINVAL, max_value=MAXVAL,
                                            sampling="linear"),
                     leaky=get_float(tuner, 'leaky', leaky, min_value=0.0, max_value=1., sampling="linear"),
                     )

    alpha = get_float(tuner, 'learning rate', learning_rate, min_value=1e-5, max_value=1e-1, sampling='log')
    tmp_model.compile(
        optimizer=keras.optimizers.Adam(alpha),  # keras.optimizers.RMSprop(alpha),
        loss=loss,
        metrics=['accuracy']
    )
    return tmp_model

def get_name(fn):
    return fn.__annotations__['return'].__name__

# Confiugrations

In [6]:
config = {
    'Datasets': [
        #"CharacterTrajectories",
        #"Libras",
        "SpokenArabicDigits",
        #"ArticularyWordRecognition",
        #"Epilepsy",
        #"JapaneseVowels"
    ],
    'Classes': [
        #'Free Interconnectivity',
        'Multiple S.R.',
        'Single S.R.'
    ],
    'Models': [
        build_ESN,
        build_IRESN,
        build_IIRESN,
        build_IIRESNvsr
    ],
    'Skip': {
        get_name(build_ESN)  : ['Multiple S.R.', 'Single S.R.'],  # Experiments name of those classes must be equals
        #get_name(build_ESN1_2): ['Multiple S.R.', 'Single S.R.'],
    },
    #                Units         | Spectral radius | use G.S.R.    | off-diag       | Connectivity     | Input scaling  | Bias scaling   | leaky     | learning rate
    #                Fre-Res       | Fre-Res         | Fre-fix-res   | Fre-Res        | Fre-Res          | Fre-Res        | Fre-Res        |Fre-fix-res|Fre-fix-res
    'Free Interconnectivity': {
        'All':      (HP.free(),     HP.free(),       HP.free(),       HP.free(),       HP.free(),         HP.free(),       HP.free(),       HP.free(), HP.free())
    },
    'Multiple S.R.': {
        #    'Units': (HP.restricted(), HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        # 'Units 50': (HP.fixed(50),  HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        # 'Units 75': (HP.fixed(75),  HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        'Units 100': (HP.fixed(100), HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        #'Units 150': (HP.fixed(150), HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        #'Units 250': (HP.fixed(250), HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
    },
    'Single S.R.': {
        #    'Units': (HP.restricted(),  HP.free(),       HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        # 'Units 50': (HP.fixed(50),  HP.restricted(), HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        # 'Units 75': (HP.fixed(75),  HP.restricted(), HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        'Units 100': (HP.fixed(100), HP.restricted(), HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        #'Units 150': (HP.fixed(150), HP.restricted(), HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
        #'Units 250': (HP.fixed(250), HP.restricted(), HP.fixed(False), HP.restricted(), HP.restricted(1.), HP.restricted(), HP.restricted(), HP.free(), HP.free()),
    }
}

# Run all

In [7]:

from datetime import timedelta

datasets = config.get('Datasets')
classes = config.get('Classes')
models_fn = config.get('Models')
skip = config.get('Skip')

run_time = time()
for dataset_name in datasets:
    train_path = os.path.join(DATA_ROOT, dataset_name, dataset_name + '_TRAIN.ts')
    test_path = os.path.join(DATA_ROOT, dataset_name, dataset_name + '_TEST.ts')

    x_train_all, y_train_all = load_sktime_dataset(train_path)
    x_test, y_test = load_sktime_dataset(test_path)

    x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all,
                                                      test_size=0.33, random_state=42, stratify=y_train_all)

    train_set = (x_train, y_train)
    val_set = (x_val, y_val)
    test_set = (x_test, y_test)

    features = x_train.shape[-1]
    output_units = len(np.unique(y_test))  # Dataset must have one of each features

    for class_name in classes:
        for experiment_name, params in config.get(class_name).items():
            for model_fn in models_fn:
                model_name = get_name(model_fn)
                print("[{}] M.S. of {: >25} {: >15} {: >10} {: >10}".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S"), dataset_name, class_name, experiment_name,
                                                              model_name))
                already_tested = benchmarks.is_benchmarked(dataset_name, class_name, experiment_name, model_name)
                if already_tested and SKIP:
                    print("[                   ] Skip Already tested!")
                    continue
                start_model = time()
                build_fn = partial(model_fn, output_units, features, *params)
                names = (dataset_name, class_name, experiment_name, model_name)

                tuner, score = model_selection(build_fn, names,
                                        train_set, val_set,
                                        tuner_path=TUNER_ROOT, verbose=0)

                duration = time() - start_model
                string_out = "["+datetime.now().strftime("%d/%m/%Y %H:%M:%S") +"] M.S. run time " + str(timedelta(seconds=duration))
                print(string_out)

                if BENCHMARKS_TRIALS > 0:
                    model, stat = testing_model(names, tuner, train_set, val_set, test_set)
                    stat.add_score(score)
                    benchmarks.add(dataset_name, class_name, experiment_name, model_name, stat)
                    plot_model(model, names, path=WEIGHTS_ROOT, show=False)

                if model_name in skip.keys():  # Il modello è nei duplicati
                    equals_classes = skip[model_name].copy()
                    if class_name in equals_classes:  # Anche la classe è nei duplicati
                        equals_classes.remove(class_name)
                        stat = benchmarks.get(dataset_name, class_name, experiment_name, model_name)
                        if stat is not None:
                            benchmarks.copy_stats(dataset_name, equals_classes, experiment_name, model_name, stat)

                        source_dir = os.path.join(TUNER_ROOT, TUNER_STRING, dataset_name, class_name, experiment_name + ' ' + model_name)
                        for to_copy_classes in equals_classes:
                            dest_dir = os.path.join(TUNER_ROOT, TUNER_STRING, dataset_name, to_copy_classes)
                            if not os.path.exists(dest_dir):
                                os.makedirs(dest_dir)
                            dest_dir = os.path.join(dest_dir, experiment_name + ' ' + model_name)
                            if not os.path.exists(dest_dir):
                                os.symlink(source_dir, dest_dir)
                benchmarks.save()

benchmarks.save()

duration = time() - run_time
string_out = "Requested time: " + str(timedelta(seconds=duration))
print(string_out)
send_notification("All Done", string_out)

/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/sktime/utils/data_io.py:63: FutureWarning: This function has moved to datasets/_data_io, this version will be removed in V0.10
  warn(
/dati/luca/Uni-Luca/Tesi/progetto/venv/lib/python3.9/site-packages/sktime/utils/data_io.py:63: FutureWarning: This function has moved to datasets/_data_io, this version will be removed in V0.10
  warn(


[17/03/2022 11:48:40] M.S. of        SpokenArabicDigits   Multiple S.R.  Units 100        ESN
[                   ] Skip Already tested!
[17/03/2022 11:48:40] M.S. of        SpokenArabicDigits   Multiple S.R.  Units 100     IIRESN
[                   ] Skip Already tested!
Requested time: 0:00:09.049946
